In [1]:
import numpy as np
import base64
import os
from detectron2.config import get_cfg
from flask import Flask,render_template,request
import io
from PIL import Image
from detectron2.utils.visualizer import Visualizer,ColorMode
from detectron2.data import MetadataCatalog,DatasetCatalog
from detectron2.engine import DefaultPredictor
import cv2 as cv

In [4]:
cfg=get_cfg()
cfg.merge_from_file(os.path.join('output_models','config.yaml'))
#cfg.MODEL.WEIGHTS=os.path.join('output_models','model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

c:\Users\deyar\OneDrive\Desktop\food detection\detectron2_env\lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.l

In [2]:
import pickle

In [5]:
file_path=os.path.join(cfg.OUTPUT_DIR, "meta_data.pkl")
with open(file_path,'rb') as file_obj:
  meta_data=pickle.load(file_obj)

In [6]:
meta_data

namespace(name='food',
          json_file='json\\_annotations.coco.json',
          image_root='data',
          evaluator_type='coco',
          thing_classes=['burger', 'pizza'],
          thing_dataset_id_to_contiguous_id={1: 0, 2: 1})

In [7]:
meta_data.get('thing_classes')

['burger', 'pizza']

In [8]:
app=Flask(__name__,template_folder='demo/')
cfg=get_cfg()
cfg.merge_from_file(os.path.join('output_models','config.yaml'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)
UPLOAD_FOLDER = 'uploads/'
app.config['UPLOAD_FOLDER']=UPLOAD_FOLDER
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
@app.route ('/')
def home():
    return render_template('test.html')
@app.route('/pred', methods=['POST'])
def marks():
    file=request.files['image']
    file.save(os.path.join('uploads',file.filename))
    file_path=os.path.join('uploads',file.filename)
    image=cv.imread(file_path) #load image
    output=predictor(image)
    v=Visualizer(image,metadata=meta_data,scale=.3,instance_mode=ColorMode.IMAGE)
    out=v.draw_instance_predictions(output['instances'].to('cpu'))
    img=out.get_image()[:,:,::-1]
    segmented_image=Image.fromarray(img) #convert to pil image
    segmented_image=segmented_image.resize((400,400)) #resize
    buffered = io.BytesIO()
    segmented_image.save(buffered, format="JPEG")
    buffered.seek(0)
    img_str = base64.b64encode(buffered.getvalue()).decode() #encoding 
    image = Image.open(file.stream) #load image
    image=image.resize((400,400)) #resize
    np_array=np.array(image) #convert into numpy array
    pil_image = Image.fromarray(np_array) #again convert into PIL image
    buffered = io.BytesIO() #create object
    pil_image.save(buffered, format="JPEG") #save in bytes    
    plot_url1= base64.b64encode(buffered.getvalue()).decode()
    return render_template('res.html',x=img_str,y=plot_url1)

In [9]:
if __name__=='__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Sep/2024 20:39:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 20:39:46] "GET /favicon.ico HTTP/1.1" 404 -
c:\Users\deyar\OneDrive\Desktop\food detection\detectron2_env\lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
127.0.0.1 - - [02/Sep/2024 20:39:58] "POST /pred HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 20:40:32] "POST /pred HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 20:41:13] "POST /pred HTTP/1.1" 200 -
